In [ ]:
! pip install transformers peft datasets

In [2]:
from huggingface_hub import notebook_login,login
notebook_login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PrefixTuningConfig, TaskType, PeftType
import torch
import os
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm

model_name='bigscience/bloomz-560m'
tokenizer_name='bigscience/bloomz-560m'
peft_config=PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)

dataset_name='twitter_complaints'
checkpoing_name=f"{dataset_name}_{model_name}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace('/','_')
text_column='Tweet text'
label_column='text_label'
max_length=64
lr=3e-2
num_epochs=50
batch_size=8

In [6]:
dataset=load_dataset('ought/raft',dataset_name)
# Extract and format class names
classes=[k.replace('_',' ')for k in dataset['train'].features['Label'].names]
# Map label IDs to text labels
dataset=dataset.map(lambda x:{'text_label':[classes[label]for label in x['Label']]},batched=True,num_proc=1   )

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [8]:
# Data Preprocessing
tokenizer=AutoTokenizer.from_pretrained(model_name)
# Ensures padding tokens exists
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id=tokenizer.eos_token_id
# Computes max token length for class labels
target_max_length=max([len(tokenizer(class_label)['input_ids'])for class_label in classes])

def preprocess_function(examples):
  # Get batch size from input
  batch_size=len(examples[text_column])
  # Format input prompt
  inputs= [f"{text_column}: {x} Label :" for x in examples[text_column]]
  # Target labels
  targets=[str(x) for x in examples[label_column]]
  # TOkenizer input prompt
  model_inputs=tokenizer(inputs)
  labels=tokenizer(targets, add_special_tokens=False)
  for i in range(batch_size):
    sample_input_ids=model_inputs['input_ids'][i]
    label_input_ids=labels['input_ids'][i] + [tokenizer.eos_token_id]

    model_inputs['input_ids'][i] = sample_input_ids + label_input_ids
    labels['input_ids'][i] =[-100] * len(sample_input_ids) + label_input_ids  #  ignore the input tokens, Focus only generated label part
    model_inputs['attention_mask'][i] = [i] * len(model_inputs['input_ids'][i])
  # Ensures all sequences are padded to the same length
  for i in range(batch_size):
    sample_input_ids=model_inputs['input_ids'][i]
    label_input_ids =labels['input_ids'][i]
    # Left pad input IDs with pad_token_id
    model_inputs['input_ids'][i] = [tokenizer.pad_token_id] * (max_length -len(sample_input_ids)) + sample_input_ids
    # Adjust attention mask to match padding
    model_inputs['attention_mask'][i] =[0] * (max_length - len(sample_input_ids)) + model_inputs['attention_mask'][i]
    # Ignore the padded tokens
    labels['input_ids'][i] =[-100] * (max_length - len(sample_input_ids)) + label_input_ids
    model_inputs['input_ids'][i] = torch.tensor(model_inputs['input_ids'][i][:max_length])
    model_inputs['attention_mask'][i]=torch.tensor(model_inputs['attention_mask'][i][:max_length])
    labels['input_ids'][i] = torch.tensor(labels['input_ids'][i][:max_length])
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

processed_datasets=dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset['train'].column_names,
    load_from_cache_file=False,
    desc='Running tokenizer on dataset'
)
train_dataset=processed_datasets['train']
eval_dataset=processed_datasets['train']

train_dataloader=DataLoader(train_dataset,shuffle=True, collate_fn=default_data_collator, batch_size=batch_size,pin_memory=True)
eval_dataloader=DataLoader(eval_dataset,collate_fn=default_data_collator, batch_size=batch_size,pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [10]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs


test_dataset = dataset["test"].map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)


Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

## Actually we are training the Trainable parameters on our datast. Which will help to LLM behaviour of the data.

In [11]:
# Creating Model
model=AutoModelForCausalLM.from_pretrained(model_name)
model=get_peft_model(model,peft_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

trainable params: 1,474,560 || all params: 560,689,152 || trainable%: 0.2630


In [12]:
model.peft_config

{'default': PrefixTuningConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path='bigscience/bloomz-560m', revision=None, inference_mode=False, num_virtual_tokens=30, token_dim=1024, num_transformer_submodules=1, num_attention_heads=16, num_layers=24, encoder_hidden_size=1024, prefix_projection=False)}

## Now we are doing Training and BackPropagation of the Trainable parameter. So that it adjust its value according to accuracy.

In [14]:
# model
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
# Training and Evaluation
for epoch in range(num_epochs):
  model.train()
  total_loss=0
  for step,batch in enumerate(tqdm(train_dataloader)):
    batch ={k: v for k,v in batch.items()}
    outputs=model(**batch)
    loss=outputs.loss
    total_loss += loss.detach().float()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  model.eval()
  eval_loss=0
  eval_preds=[]
  for step, batch in enumerate(tqdm(eval_dataloader)):
    batch ={k:v for k, v in batch.items()}
    with torch.no_grad():
      outputs=model(**batch)
    loss=outputs.loss
    eval_loss += loss.detach().float()
    eval_preds.extend(tokenizer.batch_decode(torch.argmax(outputs.logits,-1).detach().cpu().numpy(),skip_special_tokens=True))


    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

 14%|█▍        | 1/7 [00:08<00:52,  8.80s/it]

epoch=0: train_ppl=tensor(1073.8441) train_epoch_loss=tensor(6.9790) eval_ppl=tensor(1.3376) eval_epoch_loss=tensor(0.2909)


 29%|██▊       | 2/7 [00:17<00:44,  8.80s/it]

epoch=0: train_ppl=tensor(1073.8441) train_epoch_loss=tensor(6.9790) eval_ppl=tensor(2.3518) eval_epoch_loss=tensor(0.8552)


 43%|████▎     | 3/7 [00:25<00:33,  8.34s/it]

...

 86%|████████▌ | 6/7 [00:50<00:08,  8.16s/it]

epoch=27: train_ppl=tensor(98.3378) train_epoch_loss=tensor(4.5884) eval_ppl=tensor(27.6148) eval_epoch_loss=tensor(3.3184)


100%|██████████| 7/7 [00:54<00:00,  7.73s/it]


epoch=27: train_ppl=tensor(98.3378) train_epoch_loss=tensor(4.5884) eval_ppl=tensor(114.7623) eval_epoch_loss=tensor(4.7429)


 43%|████▎     | 3/7 [01:45<02:21, 35.27s/it]

In [ ]:
model.eval()
i=16
inputs = tokenizer(f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : ', return_tensors="pt")

with torch.no_grad():
  inputs={k: v for k, v in inputs.items()}
  outputs= model.generate(inputs_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'],max_new_tokens=10, eos_token_id=3)


In [ ]:
model.push_to_hub(
  f"{dataset_name}_{model_name}_{peft_config.peft_type}_{peft_config.task_type}".replace("/", "_"))

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{dataset_name}_{model_name}_{peft_config.peft_type}_{peft_config.task_type}".replace(
    "/", "_"
)

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:

model.eval()
i = 4
inputs = tokenizer(f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : ', return_tensors="pt")
print(dataset["test"][i]["Tweet text"])
print(inputs)

with torch.no_grad():
    inputs = {k: v for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))